# Project 4 - Part I

# S C R A P I N G

<div class="alert alert-block alert-info">
<b>Import all required libraries and folders</b> 
</div>

In [1]:
#from bs4 import BeautifulSoup
import bs4
import requests
import urllib.request, urllib.parse, urllib.error

# import sleep
from time import sleep

# Regular Expression
import re

# Pandas and Numpy
import pandas as pd 
import numpy as np

# import logging

# import nltk

<div class="alert alert-block alert-info">
<b>Selection Information</b> 
</div>

In [2]:
class Job:
    
    def __init__(self, url):
        seek = requests.get(url)
        if seek.status_code != 200:
            logging.error("Status code was " + str(seek.status_code))
            raise ValueError
        self.html = seek.text
        self.soup = bs4.BeautifulSoup(self.html, 'lxml')
          
    def title(self):
        try:
            title_extract = self.soup.find('section', attrs={'aria-labelledby':'jobHeader'}).find(attrs={'data-automation':'job-detail-title'}).text
            return title_extract
            
        except:
            return np.nan
        
    def advertiser(self):
        try:
            advertiser_extract = self.soup.find('section', attrs={'aria-labelledby':'jobHeader'}).text.split('from')[1]
            return advertiser_extract
        except:
            return np.nan
       
    def description(self):
        try:
            description_extract = self.soup.find_all(class_='templatetext')[0].text
            return description_extract
        except:
            return np.nan

    def date(self):
        try:
            date_extract = self.soup.find('span', class_="Eadjc1o", text='Job Listing Date').parent.parent.find('dd').text
            return date_extract
        except:
            return np.nan
    
    def salary(self):
        try:
            salary_extract = self.soup.find('span', class_="Eadjc1o", text='Salary').parent.parent.find('dd').text
            return salary_extract
        except:
            return np.nan
       
    def worktype(self):
        try:
            worktype_extract = self.soup.find('span', class_="Eadjc1o", text='Work Type').parent.parent.find(text=['Full Time','Part Time','Contract/Temp','Casual/Vacation'])
            return worktype_extract
        except:
            return np.nan
       
    def classification(self):
        try:
            classificaiton_extract = self.soup.find('span', class_="Eadjc1o", text='Classification').parent.parent.find('strong').text
            return classificaiton_extract 
        except:
            return np.nan
   
    def sub_classification(self):
        try:
            sub_classificaiton_extract = self.soup.find('span', class_="Eadjc1o", text='Classification').parent.parent.find(class_='lwHBT6d').text
            return sub_classificaiton_extract.replace(',','',1).strip()
        except:
            return np.nan
       
    def city(self):
        try:
            city_extract = self.soup.find('span', class_="Eadjc1o", text='Location').parent.parent.find('strong').text
            return city_extract
        except:
            return np.nan
        
    def suburb(self):
        try:
#            suburb_extract = self.soup.find('span', class_="Eadjc1o", text='Location').parent.parent.find('span', class_="lwHBT6d").text 
            suburb_extract = self.soup.find('span', class_="Eadjc1o", text='Location').parent.parent.find('strong').text
            return suburb_extract.replace(',','',1).strip()
        except:
            return np.nan
                
    def next_page_check(self):
        try:
            next_page_check = self.soup.find('a', class_="Eadjc1o", text='Location').parent.parent.find('span', class_="lwHBT6d").text 
            return suburb_extract.replace(',','',1).strip()
        except:
            return np.nan

<div class="alert alert-block alert-info">
<b>Scraping through pages and grabbing information</b> 
</div>

In [5]:
def get_job_links_on_page(url):
    page_number = 0
    while True and page_number < 2:         # Just select 1 Page of each category
#    while True and page_number < 41:         # Just select 40 Pages of each category
#    while True: 

#        print ('Scraping\t', (url + "?page=" + str(page_number)))
        seek = requests.get(url + "?page=" + str(page_number))
        if seek.status_code != 200:
             raise ValueError
        soup = bs4.BeautifulSoup(seek.text)
        fail = soup.find(string='Sorry, we couldn’t find anything. Maybe try taking the following steps…')
#         print (seek.text)
#         break
        if fail == None:
            for link in soup.find_all('a', class_ = '_2iNL7wI',attrs={'data-automation':'jobTitle'}): 
                yield 'https://www.seek.com.au' + link['href']
            page_number += 1
        else:
            print ("+"*30)
            print ('finished' + url + 'on page' + str(page_number-1))
            break

<div class="alert alert-block alert-info">
<b>List of Job Titles and Skills to scrape from SEEK</b> 
</div>

In [6]:
# Select Titles and Skills
selection_list = ['Data-Science',
                  'Data-Scientist',
                  'Data-Analyst',
                  'Data-Engineer',
                  'Data-Architect',
                  'Research-Scientist',
                  'Business-Intelligence-Data',
                  'BI-Data',
                  'Business-Analyst-Data',
                  'BA-Data',
    
                 # Skills Required
                  'Hadoop', 'Machine-Learning', 'Deep-Learning', 'Big-Data', 'Tensorflow',
                  'Keras', 'Theanos', 'Scikit', 'Numpy', 'ETL']

<div class="alert alert-block alert-info">
<b>Start Scrapping SEEK Website</b> 
</div>

In [7]:
import time

all_jobs = []
all_urls = []
job_cat = []
job_categories = {c:0 for c in selection_list}
link_counter = 0

for c in selection_list:
    url = 'https://www.seek.com.au/' + str(c) + str('-jobs')
    cat_counter = 0
    print (c)
    for link in get_job_links_on_page(url):
        print (link, link_counter)
        try:
            all_jobs.append(Job(link))
            all_urls.append(link)
            link_counter +=1
            job_cat.append(c)
            cat_counter += 1
            job_categories[c] = cat_counter
        except:
            print ('JOB has been removed or Page not found')
        time.sleep(0.1)

Data-Science
https://www.seek.com.au/job/38473593?type=promoted&searchrequesttoken=039a7cbc-affd-4343-94f9-fdc88eb37885 0
https://www.seek.com.au/job/38337813?type=promoted&searchrequesttoken=039a7cbc-affd-4343-94f9-fdc88eb37885 1
https://www.seek.com.au/job/38549522?type=standout&searchrequesttoken=039a7cbc-affd-4343-94f9-fdc88eb37885 2
https://www.seek.com.au/job/38547195?type=standard&searchrequesttoken=039a7cbc-affd-4343-94f9-fdc88eb37885 3
https://www.seek.com.au/job/38508350?type=standout&searchrequesttoken=039a7cbc-affd-4343-94f9-fdc88eb37885 4
https://www.seek.com.au/job/38514752?type=standout&searchrequesttoken=039a7cbc-affd-4343-94f9-fdc88eb37885 5
https://www.seek.com.au/job/38551383?type=standout&searchrequesttoken=039a7cbc-affd-4343-94f9-fdc88eb37885 6
https://www.seek.com.au/job/38444083?type=standout&searchrequesttoken=039a7cbc-affd-4343-94f9-fdc88eb37885 7
https://www.seek.com.au/job/38520969?type=standard&searchrequesttoken=039a7cbc-affd-4343-94f9-fdc88eb37885 8
https:

https://www.seek.com.au/job/38523726?type=standout&searchrequesttoken=a1985f91-26b5-4534-88d3-6fe374dcabd6 75
https://www.seek.com.au/job/38507417?type=standout&searchrequesttoken=a1985f91-26b5-4534-88d3-6fe374dcabd6 76
https://www.seek.com.au/job/38556858?type=standard&searchrequesttoken=a1985f91-26b5-4534-88d3-6fe374dcabd6 77
https://www.seek.com.au/job/38556593?type=standout&searchrequesttoken=a1985f91-26b5-4534-88d3-6fe374dcabd6 78
https://www.seek.com.au/job/38502353?type=standard&searchrequesttoken=a1985f91-26b5-4534-88d3-6fe374dcabd6 79
https://www.seek.com.au/job/38518667?type=standout&searchrequesttoken=a1985f91-26b5-4534-88d3-6fe374dcabd6 80
https://www.seek.com.au/job/38461920?type=standard&searchrequesttoken=a1985f91-26b5-4534-88d3-6fe374dcabd6 81
https://www.seek.com.au/job/38557632?type=standard&searchrequesttoken=a1985f91-26b5-4534-88d3-6fe374dcabd6 82
https://www.seek.com.au/job/38490404?type=standard&searchrequesttoken=a1985f91-26b5-4534-88d3-6fe374dcabd6 83
https://ww

https://www.seek.com.au/job/38521768?type=standard&searchrequesttoken=fb467302-8c95-4a43-bf48-2fd66db71403 149
https://www.seek.com.au/job/38534827?type=standout&searchrequesttoken=fb467302-8c95-4a43-bf48-2fd66db71403 150
https://www.seek.com.au/job/38548563?type=standard&searchrequesttoken=fb467302-8c95-4a43-bf48-2fd66db71403 151
https://www.seek.com.au/job/38490439?type=standout&searchrequesttoken=fb467302-8c95-4a43-bf48-2fd66db71403 152
https://www.seek.com.au/job/38521862?type=standard&searchrequesttoken=fb467302-8c95-4a43-bf48-2fd66db71403 153
https://www.seek.com.au/job/38500269?type=promoted&searchrequesttoken=06ee347d-bb60-4039-b4f4-449b885f822c 154
https://www.seek.com.au/job/38485319?type=promoted&searchrequesttoken=06ee347d-bb60-4039-b4f4-449b885f822c 155
https://www.seek.com.au/job/38543523?type=standard&searchrequesttoken=06ee347d-bb60-4039-b4f4-449b885f822c 156
https://www.seek.com.au/job/38531646?type=standout&searchrequesttoken=06ee347d-bb60-4039-b4f4-449b885f822c 157
h

https://www.seek.com.au/job/38531673?type=standout&searchrequesttoken=71d52acd-ac1a-4d1b-94e5-cfb5c26e3958 223
https://www.seek.com.au/job/38470931?type=standout&searchrequesttoken=71d52acd-ac1a-4d1b-94e5-cfb5c26e3958 224
https://www.seek.com.au/job/38535680?type=standard&searchrequesttoken=71d52acd-ac1a-4d1b-94e5-cfb5c26e3958 225
https://www.seek.com.au/job/38447334?type=standout&searchrequesttoken=71d52acd-ac1a-4d1b-94e5-cfb5c26e3958 226
https://www.seek.com.au/job/38470621?type=standard&searchrequesttoken=71d52acd-ac1a-4d1b-94e5-cfb5c26e3958 227
https://www.seek.com.au/job/38551044?type=standout&searchrequesttoken=71d52acd-ac1a-4d1b-94e5-cfb5c26e3958 228
https://www.seek.com.au/job/38470929?type=standout&searchrequesttoken=71d52acd-ac1a-4d1b-94e5-cfb5c26e3958 229
https://www.seek.com.au/job/38470622?type=standard&searchrequesttoken=71d52acd-ac1a-4d1b-94e5-cfb5c26e3958 230
https://www.seek.com.au/job/38550026?type=standard&searchrequesttoken=71d52acd-ac1a-4d1b-94e5-cfb5c26e3958 231
h

https://www.seek.com.au/job/38486399?type=standard&searchrequesttoken=a9d96a2a-be39-4760-ada1-62b6cce8bebf 297
BI-Data
https://www.seek.com.au/job/38554063?type=standard&searchrequesttoken=f19a30d9-0107-424e-9aa0-fd15c905bbc0 298
https://www.seek.com.au/job/38553507?type=standard&searchrequesttoken=f19a30d9-0107-424e-9aa0-fd15c905bbc0 299
https://www.seek.com.au/job/38524723?type=standout&searchrequesttoken=f19a30d9-0107-424e-9aa0-fd15c905bbc0 300
https://www.seek.com.au/job/38536992?type=standout&searchrequesttoken=f19a30d9-0107-424e-9aa0-fd15c905bbc0 301
https://www.seek.com.au/job/38496941?type=standard&searchrequesttoken=f19a30d9-0107-424e-9aa0-fd15c905bbc0 302
https://www.seek.com.au/job/38545619?type=standard&searchrequesttoken=f19a30d9-0107-424e-9aa0-fd15c905bbc0 303
https://www.seek.com.au/job/38495464?type=standard&searchrequesttoken=f19a30d9-0107-424e-9aa0-fd15c905bbc0 304
https://www.seek.com.au/job/38492093?type=standout&searchrequesttoken=f19a30d9-0107-424e-9aa0-fd15c905bb

https://www.seek.com.au/job/38533705?type=standout&searchrequesttoken=fe1e3a0e-a643-47a8-9569-fc4dc6a63cfb 371
https://www.seek.com.au/job/38530948?type=standard&searchrequesttoken=fe1e3a0e-a643-47a8-9569-fc4dc6a63cfb 372
https://www.seek.com.au/job/38528568?type=standard&searchrequesttoken=fe1e3a0e-a643-47a8-9569-fc4dc6a63cfb 373
https://www.seek.com.au/job/38550545?type=standard&searchrequesttoken=fe1e3a0e-a643-47a8-9569-fc4dc6a63cfb 374
https://www.seek.com.au/job/38544684?type=standard&searchrequesttoken=fe1e3a0e-a643-47a8-9569-fc4dc6a63cfb 375
https://www.seek.com.au/job/38547329?type=standard&searchrequesttoken=fe1e3a0e-a643-47a8-9569-fc4dc6a63cfb 376
https://www.seek.com.au/job/38546684?type=standard&searchrequesttoken=fe1e3a0e-a643-47a8-9569-fc4dc6a63cfb 377
BA-Data
https://www.seek.com.au/job/38557079?type=standout&searchrequesttoken=747d275e-c1d9-4472-a1d0-e4a33b9f7e8c 378
https://www.seek.com.au/job/38552501?type=standout&searchrequesttoken=747d275e-c1d9-4472-a1d0-e4a33b9f7e

https://www.seek.com.au/job/38545070?type=standard&searchrequesttoken=d3c680d2-edbf-4628-9c2f-4eda1f52e665 445
https://www.seek.com.au/job/38517657?type=standout&searchrequesttoken=d3c680d2-edbf-4628-9c2f-4eda1f52e665 446
https://www.seek.com.au/job/38478011?type=standard&searchrequesttoken=d3c680d2-edbf-4628-9c2f-4eda1f52e665 447
https://www.seek.com.au/job/38429775?type=standout&searchrequesttoken=d3c680d2-edbf-4628-9c2f-4eda1f52e665 448
https://www.seek.com.au/job/38429760?type=standout&searchrequesttoken=d3c680d2-edbf-4628-9c2f-4eda1f52e665 449
https://www.seek.com.au/job/38460633?type=standard&searchrequesttoken=d3c680d2-edbf-4628-9c2f-4eda1f52e665 450
https://www.seek.com.au/job/38475874?type=standout&searchrequesttoken=d3c680d2-edbf-4628-9c2f-4eda1f52e665 451
https://www.seek.com.au/job/38407902?type=standout&searchrequesttoken=d3c680d2-edbf-4628-9c2f-4eda1f52e665 452
https://www.seek.com.au/job/38460126?type=standard&searchrequesttoken=d3c680d2-edbf-4628-9c2f-4eda1f52e665 453
h

https://www.seek.com.au/job/38530169?type=standard&searchrequesttoken=3fb7869c-8404-4ebd-a6a4-1fff9e91ee94 519
https://www.seek.com.au/job/38508350?type=standout&searchrequesttoken=3fb7869c-8404-4ebd-a6a4-1fff9e91ee94 520
https://www.seek.com.au/job/38551256?type=standout&searchrequesttoken=3fb7869c-8404-4ebd-a6a4-1fff9e91ee94 521
https://www.seek.com.au/job/38530822?type=standard&searchrequesttoken=3fb7869c-8404-4ebd-a6a4-1fff9e91ee94 522
https://www.seek.com.au/job/38405414?type=standard&searchrequesttoken=3fb7869c-8404-4ebd-a6a4-1fff9e91ee94 523
https://www.seek.com.au/job/38403920?type=standard&searchrequesttoken=3fb7869c-8404-4ebd-a6a4-1fff9e91ee94 524
https://www.seek.com.au/job/38353759?type=standard&searchrequesttoken=3fb7869c-8404-4ebd-a6a4-1fff9e91ee94 525
https://www.seek.com.au/job/38353752?type=standard&searchrequesttoken=3fb7869c-8404-4ebd-a6a4-1fff9e91ee94 526
https://www.seek.com.au/job/38492118?type=standard&searchrequesttoken=3fb7869c-8404-4ebd-a6a4-1fff9e91ee94 527
h

https://www.seek.com.au/job/38403920?type=standard&searchrequesttoken=05a74fb0-1538-4736-ba39-180375b0a161 593
https://www.seek.com.au/job/38549522?type=standout&searchrequesttoken=05a74fb0-1538-4736-ba39-180375b0a161 594
https://www.seek.com.au/job/38513903?type=standout&searchrequesttoken=05a74fb0-1538-4736-ba39-180375b0a161 595
https://www.seek.com.au/job/38512905?type=standout&searchrequesttoken=05a74fb0-1538-4736-ba39-180375b0a161 596
https://www.seek.com.au/job/38513017?type=standout&searchrequesttoken=05a74fb0-1538-4736-ba39-180375b0a161 597
https://www.seek.com.au/job/38508883?type=standard&searchrequesttoken=05a74fb0-1538-4736-ba39-180375b0a161 598
https://www.seek.com.au/job/38376941?type=standard&searchrequesttoken=05a74fb0-1538-4736-ba39-180375b0a161 599
https://www.seek.com.au/job/38477387?type=standard&searchrequesttoken=2673d36a-3ec3-4d16-add2-97e0f68fb2e5 600
https://www.seek.com.au/job/38523726?type=standout&searchrequesttoken=2673d36a-3ec3-4d16-add2-97e0f68fb2e5 601
h

https://www.seek.com.au/job/38557297?type=standout&searchrequesttoken=7fab458a-c76b-4bb5-b86d-9a1ae87f0e60 666
https://www.seek.com.au/job/38550410?type=standard&searchrequesttoken=7fab458a-c76b-4bb5-b86d-9a1ae87f0e60 667
https://www.seek.com.au/job/38556019?type=standout&searchrequesttoken=7fab458a-c76b-4bb5-b86d-9a1ae87f0e60 668
https://www.seek.com.au/job/38516853?type=standard&searchrequesttoken=7fab458a-c76b-4bb5-b86d-9a1ae87f0e60 669
https://www.seek.com.au/job/38559780?type=standout&searchrequesttoken=7fab458a-c76b-4bb5-b86d-9a1ae87f0e60 670
https://www.seek.com.au/job/38515499?type=standout&searchrequesttoken=7fab458a-c76b-4bb5-b86d-9a1ae87f0e60 671
https://www.seek.com.au/job/38515502?type=standard&searchrequesttoken=7fab458a-c76b-4bb5-b86d-9a1ae87f0e60 672
https://www.seek.com.au/job/38486153?type=standout&searchrequesttoken=7fab458a-c76b-4bb5-b86d-9a1ae87f0e60 673
https://www.seek.com.au/job/38525233?type=standard&searchrequesttoken=7fab458a-c76b-4bb5-b86d-9a1ae87f0e60 674
h

<div class="alert alert-block alert-info">
<b>Job Selections</b> 
</div>

In [8]:
job_categories

{'Data-Science': 44,
 'Data-Scientist': 44,
 'Data-Analyst': 44,
 'Data-Engineer': 44,
 'Data-Architect': 40,
 'Research-Scientist': 42,
 'Business-Intelligence-Data': 40,
 'BI-Data': 40,
 'Business-Analyst-Data': 40,
 'BA-Data': 40,
 'Hadoop': 40,
 'Machine-Learning': 40,
 'Deep-Learning': 40,
 'Big-Data': 42,
 'Tensorflow': 40,
 'Keras': 16,
 'Theanos': 0,
 'Scikit': 10,
 'Numpy': 18,
 'ETL': 44}

<div class="alert alert-block alert-info">
<b>Job reference ID</b> 
</div>

In [9]:
all_jobs;

<div class="alert alert-block alert-info">
<b>Populate all Columns</b> 
</div>

In [10]:
job_df = pd.DataFrame({
    'title': [x.title() for x in all_jobs],
    'description': [x.description() for x in all_jobs],
    'advertiser': [x.advertiser() for x in all_jobs],
    'date': [x.date() for x in all_jobs],
    'salary' : [x.salary() for x in all_jobs],
    'worktype' : [x.worktype() for x in all_jobs],
    'classificaiton' : [x.classification() for x in all_jobs],
    'sub_classificaiton' : [x.sub_classification() for x in all_jobs],
    'city': [x.city() for x in all_jobs],
    'suburb': [x.suburb() for x in all_jobs],
    'link': [x for x in all_urls],
    'category': [x for x in job_cat]})

job_df;

In [11]:
job_df.shape

(708, 12)

In [12]:
job_df.isnull().sum()

title                   0
description           152
advertiser              8
date                    0
salary                403
worktype                0
classificaiton          0
sub_classificaiton      0
city                    0
suburb                  0
link                    0
category                0
dtype: int64

In [13]:
job_df[job_df.description.isnull()];

<div class="alert alert-block alert-info">
<b>Put Output to a DataFrame</b> 
</div>

In [14]:
job_df.to_csv('Datasets/SEEK_Scrapped_Data.csv', encoding='utf8')

<div class="alert alert-block alert-info">
<b>Check to see if file was Created by reading and printing the first 5 records </b> 
</div>

In [15]:
df = pd.read_csv('Datasets/SEEK_Scrapped_Data.csv', index_col=0)
df.head();